In [1]:
import tensorflow as tf
import numpy as np
import glob
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPool2D,AvgPool2D
from tensorflow import keras
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd

In [ ]:
model=tf.keras.models.load_model("/content/drive/MyDrive/MLSP_A2/FrameWise_Bi_LSTM_Modified_2_1999")

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.7/dist-packages/keras/backend.py", line 4771, in <genexpr>
    for ta, out in zip(output_ta_t, flat_output))  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/tf_should_use.py", line 245, in wrapped
    error_in_function=error_in_function)


In [ ]:
def normalise(arr):
  mu=np.mean(arr,axis=0)
  std_dav=np.std(arr,axis=0)
  normalised_arr=(arr-mu)/std_dav
  return normalised_arr

In [ ]:
def multihot_to_indices(arr):
  non_zero_indices=[]
  for index in range (len(arr)):
    if arr[index]!=0:
      non_zero_indices.append(index)
  return non_zero_indices

In [ ]:
X_test=sorted(glob.glob("/content/drive/MyDrive/MLSP_A2/test/X/*.npy"))

In [ ]:
events_types = {
    0: 'Alarm_bell_ringing', 
    1: 'Blender', 
    2: 'Cat', 
    3: 'Dishes', 
    4: 'Dog',
    5: 'Electric_shaver_toothbrush', 
    6: 'Frying', 
    7: 'Running_water',
    8: 'Silence', 
    9: 'Speech', 
    10: 'Vacuum_cleaner'
}

In [ ]:
def prediction_to_csv(X_path):
  
  fileid=[]
  classes=[]  
  for file_no in range(len(X_path)):
   fileid.append(X_path[file_no][38:])
   events_in_this_file=[]
   #print(X_path[file_no])
   a=np.squeeze(np.load(str(X_path[file_no]))).T
   a=a.reshape(20,50,64)
   a=normalise(a)
   y_pred=model(a)
   y_pred=y_pred.numpy()
   y_pred=y_pred.reshape(1000,11)
   y_pred=np.round(y_pred)
   y_pred=y_pred.T
   y_pred=np.sum(y_pred,axis=-1)
   #print(y_pred.shape)
   #print(y_pred)
   non_zero_indices_y_pred=multihot_to_indices(y_pred)
   #print(non_zero_indices_y_pred)
   for indices in non_zero_indices_y_pred:
     events_in_this_file.append(events_types[indices])
   l=", ".join([str(item) for item in events_in_this_file])  
   classes.append(l)
  return fileid,classes 


In [ ]:
fileid,classes=prediction_to_csv(X_test)

In [ ]:
prediction=pd.DataFrame({
    'Fileid':fileid,
    'Prediction':classes
})

In [ ]:
prediction.to_csv('/content/drive/MyDrive/MLSP_A2/Predictions.csv')

In [ ]:
'''
x=np.squeeze(np.load("/content/drive/MyDrive/MLSP_A2/X/melspec_1700.npy")).T
x=x.reshape(20,50,64)
x=normalise(x)
y_pred=model(x)
y_pred=y_pred.numpy()
y_pred=y_pred.reshape(1000,11)
print(y_pred)
y_pred=np.round(y_pred)
y_pred=y_pred.T
y_pred=np.sum(y_pred,axis=-1)
print(y_pred)
y=np.load('/content/drive/MyDrive/MLSP_A2/Y/eventroll_1700.npy')
y=np.sum(y,axis=-1)
'''